In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import sys
import pymde
import scipy.sparse as sp

%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import plotly.express as px

from sklearn.cluster import DBSCAN

from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestNeighbors

sys.path.insert(1, '..')

from repsys.dataset import Dataset
from repsys.model import Model
from repsys.evaluators import DatasetEvaluator, ModelEvaluator
import repsys.dtypes as dtypes

In [4]:
class MovieLens(Dataset):
    def name(self):
        return "movielens"

    def get_genres(self):
        return self.tags.get('genres')

    def item_cols(self):
        return {
            "movieId": dtypes.ItemID(),
            "title": dtypes.Title(),
            "genres": dtypes.Tag(sep="|"),
            "year": dtypes.Number(data_type=int),
        }

    def interaction_cols(self):
        return {
            "movieId": dtypes.ItemID(),
            "userId": dtypes.UserID(),
            "rating": dtypes.Interaction(),
        }

    def load_items(self):
        df = pd.read_csv("./datasets/ml-sm/movies.csv")
        df["year"] = df["title"].str.extract("\((\d+)\)")
        return df

    def load_interactions(self):
        return pd.read_csv("./datasets/ml-sm/ratings.csv")

In [5]:
dataset = MovieLens()
dataset.split()

In [8]:
X = dataset.get_train_data()
G = X.T.dot(X).toarray()
diagIndices = np.diag_indices(G.shape[0])
G[diagIndices] += 0.5
P = np.linalg.inv(G)
B = P / (-np.diag(P))
B[diagIndices] = 0
pred = X.dot(B)

In [11]:
dataset.get_validation_data()

<46x9396 sparse matrix of type '<class 'numpy.float64'>'
	with 3978 stored elements in Compressed Sparse Row format>

In [13]:
X.shape

(518, 9396)

In [14]:
B.shape

(9396, 9396)

In [18]:
pred = dataset.get_validation_data()[0].dot(B)

In [20]:
pred.shape

(46, 9396)

In [5]:
class KNN(Model):
  def __init__(self, k=5):
    self.model = NearestNeighbors(n_neighbors=k, metric="cosine")

  def name(self):
    return "knn"

  def fit(self):
    self.model.fit(self.dataset.get_train_data())

  def predict(self, X, **kwargs):
    # the slowest phase of the prediction
    distances, indexes = self.model.kneighbors(X)

    n_distances = distances[:, 1:]
    n_indexes = indexes[:, 1:]

    n_distances = 1 - n_distances

    sums = n_distances.sum(axis=1)
    n_distances = n_distances / sums[:, np.newaxis]
        
    def f(dist, idx):
        A = self.dataset.get_train_data()[idx]
        D = sp.diags(dist)
        return D.dot(A).sum(axis=0)
    
    vf = np.vectorize(f, signature='(n),(n)->(m)')

    pred = vf(n_distances, n_indexes)
        
    pred[(X > 0).toarray()] = 0

    return pred

In [6]:
model = KNN(k=20)
model.update_dataset(dataset)
model.fit()

In [21]:
evaluator = ModelEvaluator()
evaluator.update_dataset(dataset)

In [30]:
pred.shape

(46, 9396)

In [26]:
true = dataset.get_validation_data()[1]

In [28]:
true.shape

(46, 9396)

In [31]:
evaluator._get_metrics_results(pred, dataset.get_validation_data()[1].toarray())

{'recall@5': array([0.2 , 0.  , 0.  , 0.  , 0.2 , 0.2 , 0.2 , 0.25, 0.2 , 0.  , 0.  ,
        0.  , 0.4 , 0.2 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.2 , 0.  , 0.2 ,
        0.4 , 0.  , 0.  , 0.4 , 0.4 , 0.2 , 0.  , 0.25, 0.2 , 0.4 , 0.  ,
        0.2 , 0.2 , 0.6 , 0.  , 0.4 , 0.  , 0.  , 0.4 , 0.  , 0.2 , 0.  ,
        0.2 , 0.  ], dtype=float32),
 'recall@20': array([0.05      , 0.27272728, 0.07142857, 0.05      , 0.15      ,
        0.25      , 0.2       , 0.5       , 0.8       , 0.        ,
        0.1       , 0.1       , 0.11111111, 0.15      , 0.27272728,
        0.        , 0.08333334, 0.14285715, 0.        , 0.25      ,
        0.2       , 0.15      , 0.33333334, 0.15      , 0.        ,
        0.25      , 0.15      , 0.11111111, 0.15      , 0.5       ,
        0.125     , 0.375     , 0.1       , 0.05      , 0.15      ,
        0.25      , 0.        , 0.625     , 0.2857143 , 0.        ,
        0.2       , 0.42857143, 0.08333334, 0.16666667, 0.33333334,
        0.13333334], dtype=flo

In [9]:
evaluator.evaluate(model, split='validation')

In [10]:
evaluator.save('foo.zip')

In [15]:
evaluator.print()

Model knn:
        recall@5  recall@20  recall@50
count  46.000000  46.000000  46.000000
mean    0.294203   0.283369   0.385433
std     0.230196   0.234040   0.291111
min     0.000000   0.000000   0.000000
25%     0.200000   0.114583   0.175435
50%     0.225000   0.211111   0.303846
75%     0.400000   0.448611   0.528846
max     1.000000   1.000000   1.000000
